In [1]:
import warnings
warnings.filterwarnings('ignore')
from Model_test import *
from Load_and_Clean_data import *
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import pandas_profiling
from sklearn.decomposition import PCA

In [2]:
pd.options.display.max_columns = 90

## Load, Clean and Save


In [ ]:
# Load and clean the data
df = load_raw_data()
df = clean_data(df)

# get features and target
X,y = df_featues_target(df)

# save to a file
save_X_y(X,y)


In [3]:
#load from file
X,y = load_X_y()

In [ ]:
# pandas_profiling.ProfileReport(df)

## Train Test Split

In [4]:
X_train, X_test, y_train, y_test = scale_train_test_split(X,y)

## Base Model

In [ ]:
models, model_names = train_model(X_train, y_train)

In [ ]:
y_train_predicts = predict_all(X_train, y_train, models)

In [ ]:
y_test_predicts = predict_all(X_test, y_test, models)

In [ ]:

print('--- Train data ---')
for i,y_train_pred in enumerate(y_train_predicts):
    display_acc_and_f1_score(y_train, y_train_pred, model_names[i])
print('\n\n--- Test data ---')
for i,y_test_pred in enumerate(y_test_predicts):
    display_acc_and_f1_score(y_test, y_test_pred, model_names[i])

In [ ]:
# Decision Tree overfitting because we did not limit number of splits yet

## Iteration #2

In [ ]:
from imblearn.over_sampling import SMOTE
from collections import Counter

sm = SMOTE(random_state=109)
X_new_train, y_new_train = sm.fit_sample(X_train, y_train)



In [ ]:
print(np.unique(y_new_train, return_counts=True))
np.unique(y, return_counts=True)

In [ ]:
type(X_new_train), type(y_new_train)

In [ ]:
type(X_test), type(y_test)

In [ ]:
models, model_names = train_model(X_new_train, y_new_train)

In [ ]:
y_new_train_predicts = predict_all(X_new_train, y_new_train, models)

In [ ]:
y_new_test_predicts = predict_all(np.array(X_test), np.array(y_test), models)

In [ ]:

print('--- Train data ---')
for i, y_new_train_pred in enumerate(y_new_train_predicts):
    display_acc_and_f1_score(y_new_train, y_new_train_pred, model_names[i])
print('\n\n--- Test data ---')
for i,y_new_test_pred in enumerate(y_new_test_predicts):
    display_acc_and_f1_score(y_test, y_new_test_pred, model_names[i])

## Iteration #3

In [ ]:
len(X.columns)

In [ ]:
from sklearn.decomposition import PCA
import numpy as np

pca_1 = PCA(n_components=20)
pca_2 = PCA(n_components=40)
pca_3 = PCA(n_components=60)

principalComponents = pca_1.fit_transform(X)
principalComponents = pca_2.fit_transform(X)
principalComponents = pca_3.fit_transform(X)

print(np.sum(pca_1.explained_variance_ratio_))
print(np.sum(pca_2.explained_variance_ratio_))
print(np.sum(pca_3.explained_variance_ratio_))





# print(pca_1.explained_variance_ratio_)
# print(pca_2.explained_variance_ratio_)
# print(pca_3.explained_variance_ratio_)
# print(np.sum(pca_2.explained_variance_ratio_))
# print(np.sum(pca_3.explained_variance_ratio_))

In [ ]:
pca = PCA(n_components=2)
principalComponents = pca.fit_transform(X)
print(np.sum(pca.explained_variance_ratio_))

In [ ]:
pca_df = pd.DataFrame(principalComponents, columns= ['PCA1','PCA2'])

In [ ]:
X_train_pca, X_test_pca, y_train_pca, y_test_pca = scale_train_test_split(pca_df,y)

In [ ]:
models, model_names = train_model(X_train_pca, y_train_pca)

In [ ]:
y_train_predicts = predict_all(X_train_pca, y_train_pca, models)
y_test_predicts = predict_all(X_test_pca, y_test_pca, models)

In [ ]:
print('--- Train data ---')
for i, y_new_train_pred in enumerate(y_train_predicts):
    display_acc_and_f1_score(y_train, y_new_train_pred, model_names[i])
print('\n\n--- Test data ---')
for i,y_new_test_pred in enumerate(y_test_predicts):
    display_acc_and_f1_score(y_test, y_new_test_pred, model_names[i])

## Iteration #4 setup a pipeline

In [ ]:
pipe_svm = Pipeline([('pca', PCA(n_components=27)),
            ('clf', svm.SVC(random_state=123))])

# Set grid search params
param_grid_svm = [
  {'clf__C': [0.1, 1, 10]  , 'clf__kernel': ['linear']},
  {'clf__C': [1, 10], 'clf__gamma': [0.001, 0.01], 'clf__kernel': ['rbf']},
 ]

# Construct grid search
gs_svm = GridSearchCV(estimator=pipe_svm,
            param_grid=param_grid_svm,
            scoring='accuracy',
            cv=3, verbose=2, return_train_score = True)

# Fit using grid search
gs_svm.fit(X_train, y_train)

# Best accuracy
print('Best accuracy: %.3f' % gs_svm.best_score_)

# Best params
print('\nBest params:\n', gs_svm.best_params_)

In [5]:
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV

In [6]:

dtree_clf = DecisionTreeClassifier(criterion='gini')
pipe_tree = Pipeline([('clf', dtree_clf)])
param_grid = [{'clf__max_depth':[4,6,8]}]
grid = GridSearchCV(pipe_tree, param_grid=param_grid, 
                    cv=5, verbose=1)

In [ ]:
dtree_clf.get_params().keys()

In [7]:
grid.fit(X_train, y_train)

Fitting 5 folds for each of 3 candidates, totalling 15 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  15 out of  15 | elapsed:    3.6s finished


GridSearchCV(cv=5, error_score='raise-deprecating',
       estimator=Pipeline(memory=None,
     steps=[('clf', DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best'))]),
       fit_params=None, iid='warn', n_jobs=None,
       param_grid=[{'clf__max_depth': [4, 6, 8]}], pre_dispatch='2*n_jobs',
       refit=True, return_train_score='warn', scoring=None, verbose=1)

In [ ]:
grid.best_estimator_
grid.best_score_

In [10]:
logistic = LogisticRegression()

pipe = Pipeline([('logistic', logistic)])
C = np.logspace(-4, 4, 50)
penalty = ['l1', 'l2']

parameters = dict(logistic__C=C,
                  logistic__penalty=penalty)

clf = GridSearchCV(pipe, parameters)


In [11]:
clf.fit(X_train, y_train)

KeyboardInterrupt: 

In [ ]:
print('Best Penalty:', clf.best_estimator_.get_params()['logistic__penalty'])
print('Best C:', clf.best_estimator_.get_params()['logistic__C'])

## Columns to inspect

    - Admission_type_id
  a  - discharge_disposition_id
    - admission_source_id
    - number_of_lab_procedures
    